<a href="https://colab.research.google.com/github/alanbseo/LCAworkshop/blob/master/LCA_1_2_CoalLCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# install the BW package
!pip install brightway25

In [40]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

import bw_processing as bwp

import os
import numpy as np

In [76]:
# Exercise: CO2 emissions from 1 kWh coal-fired electricity in the US

# In our technosphere matrix, we will have four production exchanges (each of amount 1),
# and three consumption exchanges.
# Our biosphere matrix will only have one number, the emission of Iron ore from Iron mining.


# activity node
coal_mining = 101
rail_transportation = 102
electricity_generation = 103
petroleum_refining = 104 # diesel

coal = 200
co2 = 201


# Technosphere
elec_t_data = np.array([
    1,   # production of petroleum refining
    1,   # production of coal mining
    1,   # production of rail transportation
    1,   # production of electricity generation
    0.006482, # input of petroleum refining (tkm)
    0.44186, # input of coal mining
    0.46079 # input of rail transportation
])

# We need integer indices for each concept. The numbers themselves are arbitrary,
# the point is that we will use them consistently in the datapackages, and maintain a mapping from our concepts to these numbers.

elec_t_indices = np.array([
    (104, 104), # production of petroleum refining
    (101, 101), # production of coal mining
    (102, 102), # production of rail transportation
    (103, 103), # production of electricity generation
    (104, 102), # input of petroleum refining
    (101, 102), # input of coal mining
    (102, 103) # input of rail transportation
    ],
    dtype=bwp.INDICES_DTYPE
)
elec_t_flip = np.array([ False, False, False, False, True, True, True]) # Numerical sign of the inputs needs to be flipped negative

# Biosphere
elec_b_data = np.array([0,         # emission of co2 (coal mining) - ignored (could be 1.235?)
                        0.018949,  # emission of co2 (rail transporation)
                        0.99407])  # emission of co2 (electricity)
elec_b_indices = np.array([
    (201, 101), # emission of co2 (coal mining)
    (201, 102), # emission of co2 (rail transportation)
    (201, 103), # emission of co2 (electicity geneneration)
    ],
    dtype=bwp.INDICES_DTYPE
)

# Characterization matrix
elec_c_data = np.array([0, 1]) # ignore coal emission
elec_c_indices = np.array([
    (200, 200), # coal
    (201, 201), # cf of co2
    ],
    dtype=bwp.INDICES_DTYPE
)





In [78]:
# The main interface for using this library is the Datapackage class. You should use the utility functions create_datapackage and load_datapackage.
# A datapackage is a set of file objects (either in-memory or on disk) that includes a metadata file object, and one or more data resource files objects.
# The metadata file object includes both generic metadata (i.e. when it was created, the data license) and metadata specific to each data resource (how it can be used in calculations, its relationship to other data resources).

elec_dp_static = bwp.create_datapackage()

elec_dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=elec_t_indices,
    data_array=elec_t_data,
    flip_array=elec_t_flip,
)
elec_dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=elec_b_indices,
    data_array=elec_b_data,
)
elec_dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=elec_c_indices,
    data_array=elec_c_data,
)

# Base class for single and multi LCA classes
elec_lca = bc.LCA(
    demand={electricity_generation: 1},
    data_objs=[elec_dp_static],
)

elec_lca

# The LC inventory. Calculate a life cycle inventory.
elec_lca.lci()
# 1) Load LCI data, and construct the technosphere and biosphere matrices.
# 2) Build the demand array
# 3) Solve the linear system to get the supply array and life cycle inventory.

elec_lca.lcia() # Calculate the life cycle impact assessment.
# 1) Load and construct the characterization matrix
# 2) Multiply the characterization matrix by the life cycle inventory

elec_lca.score # The LCIA score as a float.
# means 1.25 iron ore * 1.1 kg iron mining * 8 kg steal

1.00280150971